In [ ]:
'''
y is the sequence of tags while x is the observed sequence of words 
'''

In [191]:
'''
General functions
read & write requires encoding='utf8' due to different languages in the files.
'''
#read the training data file 
def read_file_train(fp):
    parsed = []
    f = open(fp,'r',encoding='utf8')
    while True:
        line = f.readline() 
        #clean up the data read 
        if line == '\n':
            parsed.append(line.replace('\n',' '))
        elif line == "":
            break
        else:
            parsed.append(line.strip()) #remove \n at the end 
        
    return parsed

'''
data received shall be in dictionary term of 
data = ((x1 ,y),(x2,y),...)
fn = filename 
fp = filepath
'''
def write_file(fp,fn,data):
    with open(fp+fn, 'w',encoding='utf8') as f:
        for k in data:
            print("data received:")
            print(k[0] + ' ' + k[1]+'\n')
            f.write(k[0] + ' ' + k[1]+'\n')
    f.close()
    

# build dictionaries of count(y->x) and count(y)
def built_dict(train_data):
    '''
    data will be a array of the read train data 
    1. create tuples:count out of the y and x and put them in global_y_x
    2. if the tuples were repeated, don't add a new entry into the dictionary but increase the count 
    ''' 
    '''
    1.  from global_y_x, we have gotten count(y-> x) . 
    2. need get count(y)
    '''
    huge_count_y_x = {} #dict to contain all count(y->x)
    huge_count_y = {} #dict to contain all count(y)
    for d in train_data:
        x,y = d.split(" ") #split by spaces 
        temp_tuple = (x,y)
        
        if temp_tuple not in huge_count_y_x:
            huge_count_y_x[temp_tuple] = 1
        else:
            huge_count_y_x[temp_tuple] = huge_count_y_x[temp_tuple] + 1 
        
        if y not in huge_count_y:
            huge_count_y[y] =  1
        else: 
            huge_count_y[y] = huge_count_y[y] + 1
    return huge_count_y_x,huge_count_y 

#part 2 first part 

def calculate_emmision(huge_count_y_x, huge_count_y):
    
    '''
    Calculating the emission 
    Use the tuple (x,y) to represent the occurance of e(x|y)  
    '''
    huge_emission ={} #dict to contain all the emission probabilities 
    for seq, count_y_x in huge_count_y_x.items():
        
        emission = 0.0 #if none found, then return as 0 
        tag = seq[1]
#         print(k,v)
        count_y = huge_count_y[tag] 
        emission = count_y_x / count_y
        huge_emission[seq] = emission

    return huge_emission

# parsed_train = read_file_train("EN/train")
# print(calculate_emmision(parsed_train))

#part 2 second part 
'''
    replace words that appear < k times with #UNK# 
    modify_train_set accepts k, training data
    training data is a list returned from read_file_train
'''
def modify_train_set(k,train_data):
    modified_training_data = []
    count_dict = {}
    for td in train_data:
#         print(td)
        word,tag = td.split(" ") 
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] = count_dict[word] + 1
    #identify which words appeared less than k times 
    list_of_works_less_than_k = [] 
    for w,count in count_dict.items():
        if count < k and w != ' ': #TODO: FIX THE SPACE MESS
            list_of_works_less_than_k.append(w)
    #now replace the entries in the training set  
    for td in train_data:
        word,tag = td.split(" ")
        for words_to_be_removed in list_of_works_less_than_k:
            if word == words_to_be_removed:
                index = train_data.index(td)
                train_data[index] = '#UNK# ' + tag
    return train_data
                
#part 2 third part 
'''
using the emission data, predict the possible tags (?) 
the tuple (x,y) is key to occurance of e(x|y) 
TODO:
1. replace with #UNK# 
2. for each word w in the test_data
    3. find the tuples in huge_emissison that matches w
    4. if there's more than one, select the one with max(emission probability)
    5. if there's only 1 result, then use that tag 
    6. append the tag to y_star
7. write the results of w, tag into a tuple
8. return the array of tuples
'''
def simple_sentiment_analysis(test_data,modified_train_data,huge_emission):
    #extract the words 
    extracted_train_words = []
    for d in modified_train_data:
        x,y = d.split(" ")
        extracted_train_words.append(x)
    #check whether the word exists in train. else replace with #UNK# 
    for w in test_data:
        if w not in extracted_train_words and w != " ":
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    y_star=[] #the tags, to be appended here 
#     print("test data:\n" , len(test_data))
    for w in test_data:
        temp_dict = {} # to hold the possible results found from huge_emission 
        for key,e_value in huge_emission.items():
            if w == key[0] and w != " ": #TODO: FIX THE SPACE MESS if it matches the word
                temp_dict[key] = e_value
        #now return the tag that has the max e probability 
        best_result = max(temp_dict, key=temp_dict.get)[1]
        #append a tupple for word, tag
        y_star.append((w,best_result))
    #return the dictionary -- don't do dic cos will remove duplicates 
#     predicted_data = dict(zip(test_data, y_star))
#     print(predicted_data)
    return y_star

  

In [192]:
#code to run the file 
PART2_FN = "dev.p2.out"
EN_FP = "EN/"
FR_FP = "FR/"
CN_FP = "CN/"
SG_FP = "SG/"

'''
TODO: 
SG has a '. . . . O' , must think of better way to split
Need to fix stupid spacing in the words
'''

parsedtrainData = read_file_train(EN_FP+"train")
# print(parsedtrainData)
modifiedData = modify_train_set(3,parsedtrainData)

huge_count_y_x, huge_count_y = built_dict(modifiedData)
# print(huge_count_y_x)
huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
# print(huge_emission)
parsedtestData = read_file_train(EN_FP+"dev.in")
# print(parsedtestData)
y_star = simple_sentiment_analysis(parsedtestData,modifiedData,huge_emission)
# print(y_star)
write_file(EN_FP,"dev.p2.out",y_star)



# FPList = [EN_FP,FR_FP,CN_FP,SG_FP]

# for i in FPList:
#     parsedtrainData = read_file_train(i +"train")

#     modifiedData = modify_train_set(3,parsedtrainData)

#     huge_count_y_x, huge_count_y = built_dict(modifiedData)
#     # print(huge_count_y_x)
#     huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
#     # print(huge_emission)
#     parsedtestData = read_file_train(i + "dev.in")
#     # print(parsedtestData)
#     y_star = simple_sentiment_analysis(parsedtestData,modifiedData,huge_emission)
#     write_file(i,"dev.p2.out",y_star)

IndexError: string index out of range

In [ ]:
#trash code
'''global_y_x = {} #dictionary to keep the y and x
    #place the data into the dictionary 
    for d in data:
        x,y = d.split(" ") #split by space 
        if x not in global_y_x:
            global_y_x[x] = y '''
def test_logic(test_data,modified_train_data):
#     for w in test_data:
#         if w not in modified_train_data:
#             index = test_data.index(w)
#             test_data[index] = "#UNK#"
#         for test_words in modified_train_data:
#             if test_words != w:
#                 modified_train_data[modified_train_data.index(test_words)] = "#UNK#"
#     for i in range(len(test_data)):
#         for j in range(len(modified_train_data)):
#             if test_data[i] != modified_train_data[j]:
#                 print(test_data[i])
#                 test_data[i] = "#UNK#"
    td = {("hello",4):1,("awegawega",10):5}
    bv = max(td, key=td.get)[1]
    return bv

modified_train_data =["i","love","you"]
test_data = ["i","hate","you","bobby"]
print(test_logic(test_data,modified_train_data))